In [1]:
!pip install featuretools -q # Установка

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.1/620.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.0/231.0 kB 8.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import featuretools as ft

In [3]:
data_root = "https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/"

path = data_root + 'car_train.csv'
car_info = pd.read_csv(path)

path = data_root + 'rides_info.csv'
rides_info = pd.read_csv(path)

path = data_root + 'driver_info.csv'
driver_info = pd.read_csv(path)

path = data_root + 'fix_info.csv'
fix_info = pd.read_csv(path)

In [4]:
fix_info

,car_id,worker_id,fix_date,work_type,destroy_degree,work_duration
0,P17494612l,RJ,2020-6-20 2:14,reparking,8.0,49
1,N-1530212S,LM,2020-2-9 20:25,repair,10.0,48
2,B-1154399t,ND,2019-8-24 7:1,reparking,1.0,27
3,y13744087j,PG,2019-8-10 9:29,reparking,1.0,28
4,F12725233R,YC,2020-11-12 5:22,refuel_check,8.0,47
...,...,...,...,...,...,...
145995,j92079780S,NG,2019-1-21 14:34,reparking,1.0,36
145996,N89944460U,WM,2019-4-30 11:33,repair,2.6,13
145997,f63189062N,NY,2019-3-7 20:49,repair,5.0,14
145998,j-1712772o,IP,2018-2-18 19:10,repair,5.0,13


In [5]:
es = ft.EntitySet(id="car_data")

In [6]:
ft.list_logical_types()

,name,type_string,description,physical_type,standard_tags,is_default_type,is_registered,parent_type
0,Address,address,Represents Logical Types that contain address ...,string,{},True,True,None
1,Age,age,Represents Logical Types that contain whole nu...,int64,{numeric},True,True,Integer
2,AgeFractional,age_fractional,Represents Logical Types that contain non-nega...,float64,{numeric},True,True,Double
3,AgeNullable,age_nullable,Represents Logical Types that contain whole nu...,Int64,{numeric},True,True,IntegerNullable
4,Boolean,boolean,Represents Logical Types that contain binary v...,bool,{},True,True,BooleanNullable
5,BooleanNullable,boolean_nullable,Represents Logical Types that contain binary v...,boolean,{},True,True,None
6,Categorical,categorical,Represents Logical Types that contain unordere...,category,{category},True,True,None
7,CountryCode,country_code,Represents Logical Types that use the ISO-3166...,category,{category},True,True,Categorical
8,CurrencyCode,currency_code,Represents Logical Types that use the ISO-4217...,category,{category},True,True,Categorical
9,Datetime,datetime,Represents Logical Types that contain date and...,datetime64[ns],{},True,True,None


In [7]:
from woodwork.logical_types import Categorical, Double, Datetime, Age

es = es.add_dataframe(
    dataframe_name="cars",
    dataframe=car_info,
    index="car_id",
    logical_types={"car_type": Categorical, 'fuel_type': Categorical, 'model': Categorical}
    )

es = es.add_dataframe(
    dataframe_name="rides",
    dataframe=rides_info.drop(['ride_id'], axis=1),
    index='index',
    time_index="ride_date",
    )

es = es.add_dataframe(
    dataframe_name="drivers",
    dataframe=driver_info,
    index="user_id",
    logical_types={"sex": Categorical, "first_ride_date": Datetime, "age": Age}
    )

es = es.add_dataframe(
    dataframe_name="fixes",
    dataframe=fix_info,
    index="index",
    logical_types={"work_type": Categorical, "worker_id":Categorical}
    )
es

/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

Entityset: car_data
  DataFrames:
    cars [Rows: 2337, Columns: 10]
    rides [Rows: 739500, Columns: 14]
    drivers [Rows: 15153, Columns: 7]
    fixes [Rows: 146000, Columns: 7]
  Relationships:
    No relationships

In [8]:
es = es.add_relationship("cars", "car_id", "rides", "car_id")
es = es.add_relationship("drivers", "user_id", "rides", "user_id")
es = es.add_relationship("cars", "car_id", "fixes", "car_id")

/usr/local/lib/python3.10/dist-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type Categorical for child column car_id does not match parent column car_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/featuretools/entityset/entityset.py:414: UserWarning: Logical type Categorical for child column user_id does not match parent column user_id logical type Unknown. Changing child logical type to match parent.
  warnings.warn(


In [9]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="cars",
    max_depth=1,
)
feature_matrix.head()

,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,COUNT(rides),...,MODE(fixes.work_type),MODE(fixes.worker_id),NUM_UNIQUE(fixes.work_type),NUM_UNIQUE(fixes.worker_id),SKEW(fixes.destroy_degree),SKEW(fixes.work_duration),STD(fixes.destroy_degree),STD(fixes.work_duration),SUM(fixes.destroy_degree),SUM(fixes.work_duration)
car_id,,,,,,,,,,,,,,,,,,,,,
y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug,174,...,reparking,LR,4,33,0.835907,0.826462,2.732847,10.171884,106.7,933.0
O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug,174,...,reparking,YH,5,34,0.997276,-0.296841,2.707233,8.574733,102.1,873.0
d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick,174,...,repair,AP,5,35,0.472628,0.671481,2.978077,13.040983,130.9,915.0
u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel,174,...,repair,LM,4,34,0.492743,0.639490,3.237750,14.764994,143.0,1007.0
N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel,174,...,repair,CD,4,34,0.478043,1.341642,3.216758,12.659537,135.8,981.0


In [10]:
feature_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2337 entries, y13744087j to z-1337463D
Data columns (total 87 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   model                          2337 non-null   category
 1   car_type                       2337 non-null   category
 2   fuel_type                      2337 non-null   category
 3   car_rating                     2337 non-null   float64 
 4   year_to_start                  2337 non-null   int64   
 5   riders                         2337 non-null   int64   
 6   year_to_work                   2337 non-null   int64   
 7   target_reg                     2337 non-null   float64 
 8   target_class                   2337 non-null   category
 9   COUNT(rides)                   2337 non-null   Int64   
 10  MAX(rides.deviation_normal)    2337 non-null   float64 
 11  MAX(rides.distance)            2337 non-null   float64 
 12  MAX(rides.rating)       

In [11]:
feature_matrix[feature_matrix['model'] == "Tesla Model 3"].shape

(14, 87)

In [12]:
feature_matrix[feature_matrix['model'] == "Tesla Model 3"]['MAX(rides.speed_max)'].max()

199.3701025284091

In [13]:
feature_matrix[feature_matrix['model'] == "Tesla Model 3"]['COUNT(fixes)'].mean()

34.42857142857143

In [14]:
feature_matrix[feature_matrix['model'] == "Tesla Model 3"]['SUM(fixes.work_duration)'].max()

970.0

In [15]:
!pip install geopandas folium scikit-learn -q

In [16]:
import geopandas as gpd
from sklearn.datasets import fetch_california_housing

In [31]:
df = fetch_california_housing(as_frame=True).data
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [18]:

# создадим GeoDataFrame
gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']),
        crs=4326
    ).to_crs(epsg=3857)
gdf.head(3)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,POINT (-13607694.555 4558257.461)


In [19]:
import requests, zipfile, io

#county_fname = 'https://data.ca.gov/dataset/e212e397-1277-4df3-8c22-40721b095f33/resource/'+\
#                       'b0007416-a325-4777-9295-368ea6b710e6/download/ca-county-boundaries.zip'
county_fname = 'https://github.com/a-milenkin/Competitive_Data_Science/raw/main/data/ca-county-boundaries.zip'

r2 = requests.get(county_fname)
z2 = zipfile.ZipFile(io.BytesIO(r2.content))
z2.extractall("./ca")

In [20]:
ca_counties=gpd.read_file('./ca/CA_Counties').to_crs("EPSG:3857")
ca_counties.head(3)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,06,091,00277310,06091,Sierra,Sierra County,06,H1,G4020,None,None,None,A,2468694587,23299110,+39.5769252,-120.5219926,"POLYGON ((-13431319.751 4821511.426, -13431312..."
1,06,067,00277298,06067,Sacramento,Sacramento County,06,H1,G4020,472,40900,None,A,2499183617,76073827,+38.4500114,-121.3404409,"POLYGON ((-13490651.476 4680831.603, -13490511..."
2,06,083,00277306,06083,Santa Barbara,Santa Barbara County,06,H1,G4020,None,42200,None,A,7084000598,2729814515,+34.5370572,-120.0399729,"MULTIPOLYGON (((-13423116.772 4042044.149, -13..."


In [21]:
gdf = gpd.overlay(gdf, ca_counties[['NAME', 'geometry']], how='intersection')

In [22]:
gdf.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,NAME,geometry
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,Alameda,POINT (-13606581.360 4562487.679)
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,Alameda,POINT (-13605468.165 4559667.342)
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,Alameda,POINT (-13607694.555 4558257.461)
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,Alameda,POINT (-13608807.749 4558257.461)
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,Alameda,POINT (-13608807.749 4558257.461)


In [23]:
df = gdf[gdf['NAME'] == "Los Angeles"]

In [24]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,NAME,geometry
3826,6.1946,27.0,5.861486,0.967342,2363.0,2.661036,34.27,-118.27,Los Angeles,POINT (-13165756.176 4065114.305)
3827,4.9833,32.0,6.461078,1.239521,486.0,2.910180,34.26,-118.28,Los Angeles,POINT (-13166869.371 4063767.332)
3828,4.0080,33.0,5.191176,1.165033,1845.0,3.014706,34.26,-118.29,Los Angeles,POINT (-13167982.566 4063767.332)
3829,3.5568,37.0,4.928447,1.104712,1619.0,2.825480,34.26,-118.30,Los Angeles,POINT (-13169095.761 4063767.332)
3830,4.8100,42.0,4.979499,0.963554,1145.0,2.608200,34.26,-118.30,Los Angeles,POINT (-13169095.761 4063767.332)
...,...,...,...,...,...,...,...,...,...,...
9645,5.4011,15.0,5.385744,1.702306,785.0,1.645702,33.75,-118.11,Los Angeles,POINT (-13147945.058 3995282.330)
9646,5.0926,24.0,5.121019,1.000000,592.0,1.885350,33.75,-118.11,Los Angeles,POINT (-13147945.058 3995282.330)
9647,3.3155,21.0,4.503788,1.064394,577.0,2.185606,33.81,-118.08,Los Angeles,POINT (-13144605.473 4003318.112)
9648,10.1597,16.0,7.606936,1.121387,450.0,2.601156,34.14,-118.83,Los Angeles,POINT (-13228095.091 4047616.133)


In [25]:
from shapely.geometry import Point

In [26]:
target_point = gpd.GeoSeries([Point(-118.266667, 34.033333)], crs=4326).to_crs("EPSG:3857")

In [27]:
target_point

0    POINT (-13165385.148 4033278.712)
dtype: geometry

In [28]:
df['distance'] = df.geometry.distance(target_point[0])

In [29]:
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,NAME,geometry,distance
3826,6.1946,27.0,5.861486,0.967342,2363.0,2.661036,34.27,-118.27,Los Angeles,POINT (-13165756.176 4065114.305),31837.754406
3827,4.9833,32.0,6.461078,1.239521,486.0,2.910180,34.26,-118.28,Los Angeles,POINT (-13166869.371 4063767.332),30524.725704
3828,4.0080,33.0,5.191176,1.165033,1845.0,3.014706,34.26,-118.29,Los Angeles,POINT (-13167982.566 4063767.332),30599.061108
3829,3.5568,37.0,4.928447,1.104712,1619.0,2.825480,34.26,-118.30,Los Angeles,POINT (-13169095.761 4063767.332),30713.589954
3830,4.8100,42.0,4.979499,0.963554,1145.0,2.608200,34.26,-118.30,Los Angeles,POINT (-13169095.761 4063767.332),30713.589954
...,...,...,...,...,...,...,...,...,...,...,...
9645,5.4011,15.0,5.385744,1.702306,785.0,1.645702,33.75,-118.11,Los Angeles,POINT (-13147945.058 3995282.330),41807.677025
9646,5.0926,24.0,5.121019,1.000000,592.0,1.885350,33.75,-118.11,Los Angeles,POINT (-13147945.058 3995282.330),41807.677025
9647,3.3155,21.0,4.503788,1.064394,577.0,2.185606,33.81,-118.08,Los Angeles,POINT (-13144605.473 4003318.112),36461.383731
9648,10.1597,16.0,7.606936,1.121387,450.0,2.601156,34.14,-118.83,Los Angeles,POINT (-13228095.091 4047616.133),64328.054057


In [30]:
df['distance'].mean() / 1000

24.78694145088474

In [32]:
import folium

# создаём объект карты
m = folium.Map([37, -122], tiles='OpenStreetMap', zoom_start=6)

# Добавим точки на карту с помощью обычного apply
gdf.apply(lambda row: folium.CircleMarker(location=[row["Latitude"],row["Longitude"]],
                                          popup=row.name,
                                          radius=1,).add_to(m), axis=1)

#  Добавление LA
#folium.CircleMarker(location=[34.033333, -118.266667],
#                    radius=3, color='red').add_to(m)
m

Output hidden; open in https://colab.research.google.com to view.

In [60]:
import folium

In [72]:
params_1 = {
    'geo_data': ca_counties,
    'name': "choropleth",
    'data': ca_counties,
    'columns': ["NAME", "AWATER"],
    'key_on': "feature.properties.NAME",
    'fill_color': "YlGn",
    'fill_opacity': 0.9,
    'line_opacity': .1,
    'legend_name': "Water surface",
    }

params_2 = {'location':[34.4208, -119.698],
            'radius':5,
            'color':'red',
            'popup':'Санта Барбара'}


In [76]:
m = folium.Map([37, -122], tiles='OpenStreetMap', zoom_start=6)

# # наносим на карту округа с заливкой по площади водного пространства
folium.Choropleth(**params_1).add_to(m)

# # наносим на карту город Санта-Барбара
folium.CircleMarker(**params_2).add_to(m)

folium.GeoJson(ca_counties,
               name='District Boundary',
               style_function=lambda feature: {
                   'color': 'black',  # Цвет границ
                   'weight': 2,  # Толщина линии границы
                   'fillOpacity': 0  # Прозрачность заливки
               }
              ).add_to(m)

m